<a href="https://colab.research.google.com/github/kevinsen-git/TF_Basics/blob/main/Modelling_a_Genrated_Sequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generate a fixed sequence of numbers and try finding the simplest, most accurate model 

#### 1. Generate the sequence of 1,000 numbers
#### 2. Split into 80% training + 20% testing 
#### 3. Try various models 
#### 4. Plot the estimates
#### 5. Fine tune the models 
#### 6. Try a bit of over fitting 
#### 7. Redo with some gaps in the data
#### 8. Redo with some added noise in the data 

**[Possible Tuning Methods for models]**

1. ***Change learning rate***
2. ***Change number of epochs***
3. ***Add dense layers***
4. ***Add neurons per layer*** 
5. ***Change activation function***
6. ***Change optimizer***
7. ***Change loss function***
8. ***Use more training data***


In [1]:
# Do the imports
import tensorflow as tf 
import numpy as np 
import matplotlib as plt 
 

In [37]:
# Show versions
print("TF =",tf.__version__,"| NP =",np.__version__,"| MatPlotLib = ",plt.__version__)

#pip show tensorflow matplotlib numpy
#pip list 

TF = 2.6.0 | NP = 1.19.5 | MatPlotLib =  3.2.2


In [42]:
# Function to show names and versions of the stuff I am using
def my_modules():
  print("tf =",tf.__version__,"| np =",np.__version__,"| matplotlib = ",plt.__version__)


In [43]:
my_modules()

tf = 2.6.0 | np = 1.19.5 | matplotlib =  3.2.2
